#  Import libraries


In [10]:
import re
import math
import random
import codecs
import datetime as dt

import pandas as pd
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.max_rows', 5)


## Input parameters

In [11]:
#  data
data = "data_raw/consumption_data_and_demographics.xlsx"

#  home energy reports
her_template_filename = "data_raw/her_template.html"

image_with_bg_great = "http://evident-h2020.eu/wp-content/uploads/2023/01/happy_color.png"
image_with_no_bg_great = "http://evident-h2020.eu/wp-content/uploads/2023/01/happy_no_color.png"
image_with_bg_good = "http://evident-h2020.eu/wp-content/uploads/2023/01/good_color.png"
image_with_no_bg_good = "http://evident-h2020.eu/wp-content/uploads/2023/01/good_no_color.png"
image_with_bg_moderate = "http://evident-h2020.eu/wp-content/uploads/2023/01/sad_color.png"
image_with_no_bg_moderate = "http://evident-h2020.eu/wp-content/uploads/2023/01/sad_no_color.png"

green = "#28a745"
blue = "#17a2b8"
red = "#dc3545"

#  energy conservation tips
energy_consumption_tips = [
"Air seal your home. Sealing cracks, gaps and leaks and adding insulation can save up to 10% on home heating and cooling costs.",
"Clean or replace all filters in your home regularly. Dirty filters make your system work harder and run longer than necessary.",
"During warmer months, close blinds, shades and drapes on the sunny side of your home to help keep your home's temperature cooler and reduce the work for you AC. Open shades during cooler months to let the sun warm your home.",
"Don't peek in the oven while baking! Every time you peek, the temperature can drop 25 F, making your oven use more energy to bring the temperature back up.",
"Don't leave your electronics on all day long. Only turn on your computer, monitor, printer and fax machine when you need them."
]


##  Read data

In [12]:
df_consumptions = pd.read_excel(open(data, 'rb'), sheet_name='Consumptions')
df_demographics = pd.read_excel(open(data, 'rb'), sheet_name='Demographics')
df = pd.merge(df_consumptions, df_demographics, on ='client_id', how ='inner') # merge the two dataframes

df['consumption'] = df['consumption']/1000 #  convert consumption to kWh instead of WattHours

#  keep only the relative months - let's suppose that we are sending the home energy report on January 2023,
#  thus we will include only data from July to Dec, 2022
df = df.loc[df.date >= dt.datetime.strptime('2022-07-01', '%Y-%m-%d')].reset_index(drop=True)

display(df)


,client_id,date,consumption,region,size,heating_appliance,size_class,heating_appliance_class
0,PF32K,2022-07-01,561.367,region_A,100,air_condition,medium,electricity-based
1,PF32K,2022-08-01,912.752,region_A,100,air_condition,medium,electricity-based
...,...,...,...,...,...,...,...,...
58,TY3V9,2022-11-01,1918.725,region_A,105,air_condition,medium,electricity-based
59,TY3V9,2022-12-01,2974.712,region_A,105,air_condition,medium,electricity-based


##  Create neighbourhoods based on clients' characteristics

In [13]:
# create a new column with a unique id for each neighborhood
df['neighborhood'] = df.groupby(['region', 'size_class', 'heating_appliance_class'], sort=False).ngroup() + 1 
display(df)


,client_id,date,consumption,region,size,heating_appliance,size_class,heating_appliance_class,neighborhood
0,PF32K,2022-07-01,561.367,region_A,100,air_condition,medium,electricity-based,1
1,PF32K,2022-08-01,912.752,region_A,100,air_condition,medium,electricity-based,1
...,...,...,...,...,...,...,...,...,...
58,TY3V9,2022-11-01,1918.725,region_A,105,air_condition,medium,electricity-based,1
59,TY3V9,2022-12-01,2974.712,region_A,105,air_condition,medium,electricity-based,1


##  Calculate the consumptions for each client

In [14]:
results = pd.DataFrame() #  create a dataframe to store the results

def calculate(row):
    
    clients_df = df.loc[ #  this df will temporary store all client's neighbours
        (df['neighborhood'] == row["neighborhood"]) & #  get the clients from the same enighborhood
        (df['date'] == row["date"]) & #  get only records with the same date
        (df['client_id'] != row["client_id"])] #  exclude the current user

    efficient_clients = round(len(clients_df) * 0.3) #  calculate the number of efficient neighbours as the 30% of the most efficient clients
    clients_mean = clients_df['consumption'].mean() #  calculate the average consumption of the neighbours
    efficient_clients_mean = clients_df.sort_values(by=['consumption']).head(efficient_clients)['consumption'].mean() #  calculate the average consumption of the efficient neighbours

    #  return a pandas series to append it to the results dataframe
    return pd.Series([row["client_id"], row["date"], clients_df['client_id'].to_list(), len(clients_df['client_id'].to_list()), efficient_clients, row["consumption"], clients_mean, efficient_clients_mean], 
                     index=['client_id', 'date', 'neighboor_list', 'neighboors', 'efficient_neighboors', 'client_consumption', 'neighboors_mean_consumption', 'efficient_neighboors_mean_consumption'])


results = results.append(df.apply(lambda row : calculate(row), axis = 1))
results = results.sort_values(by=['client_id', 'date'])
display(results)


,client_id,date,neighboor_list,neighboors,efficient_neighboors,client_consumption,neighboors_mean_consumption,efficient_neighboors_mean_consumption
36,AzIYx,2022-07-01,[ECVCe],1,0,353.229,432.912,NaN
37,AzIYx,2022-08-01,[ECVCe],1,0,427.558,443.757,NaN
...,...,...,...,...,...,...,...,...
16,mhbQY,2022-11-01,"[FSXdB, RtBVF]",2,1,1237.714,1798.541,1256.772
17,mhbQY,2022-12-01,"[FSXdB, RtBVF]",2,1,1717.669,2072.611,1268.237


In [ ]:
def generate(client):
   
    her_template = codecs.open(her_template_filename, 'r', encoding="utf8").read() #  load the HER template
    client_results_df = results.loc[(df['client_id'] == client)].sort_values(by=['date']) # get all client's records and sort them by date

    no_of_efficient_neighboors = client_results_df.iloc[-1]["efficient_neighboors"] #  find the number of efficient neighbours
    client_mean_consumption = client_results_df["client_consumption"].mean() #  calculate the mean consumption for the client for the whole 6months period
    average_neighboors_mean_consumption = client_results_df["neighboors_mean_consumption"].mean() #  calculate the mean consumption for the average neighbours for the whole 6months period
    efficient_neighboors_mean_consumption = client_results_df["efficient_neighboors_mean_consumption"].mean() #  calculate the mean consumption for the efficient neighbours for the whole 6months period

    if no_of_efficient_neighboors == 0: # if there are no efficient neighbours - this might happens when neighbourhoods are very small
        efficient_neighboors_mean_consumption = client_mean_consumption
        her_template = re.sub("<!--percentange-->[\s\S]*<\/tr><!--\/percentange-->", "" , her_template) #  delete the percentange section

    #  Descriptive and injunctive feedback section
    if client_mean_consumption < efficient_neighboors_mean_consumption: #  greate performance
        her_template = her_template.replace("[great_image]", image_with_bg_great)
        her_template = her_template.replace("[callout_color]", green)
        her_template = her_template.replace("[callout_text]", str(int(math.ceil((1.0 - (client_mean_consumption/efficient_neighboors_mean_consumption))*100))) + "% less")

    elif efficient_neighboors_mean_consumption < client_mean_consumption and client_mean_consumption < average_neighboors_mean_consumption: #  good performance
        her_template = her_template.replace("[good_image]", image_with_bg_good)
        her_template = her_template.replace("[callout_color]", blue)
        her_template = her_template.replace("[callout_text]", str(int((client_mean_consumption/efficient_neighboors_mean_consumption)*100 - 100)) + "% more")

    else: #  moderate performance
        her_template = her_template.replace("[moderate_image]", image_with_bg_moderate)
        her_template = her_template.replace("[callout_color]", red)
        her_template = her_template.replace("[callout_text]", str(int((client_mean_consumption/efficient_neighboors_mean_consumption)*100 - 100)) + "% more")

    # replace the rest images with the non-color version
    her_template = her_template.replace("[great_image]", image_with_no_bg_great)
    her_template = her_template.replace("[good_image]", image_with_no_bg_good)
    her_template = her_template.replace("[moderate_image]", image_with_no_bg_moderate)
  
    #  energy consumption feedback and peer comparison section
    consumer_historic_months = ','.join(client_results_df['date'].dt.date.astype(str).to_list())
    consumer_historic_values = ','.join(client_results_df['client_consumption'].astype(str).to_list())
    consumer_historic_values_neighbours = ','.join(client_results_df['neighboors_mean_consumption'].astype(str).to_list())
    consumer_historic_values_efficient_neighbours = ','.join(client_results_df['efficient_neighboors_mean_consumption'].astype(str).to_list())
    
    her_template = her_template.replace("[[historic_period_months]]", str(consumer_historic_months))
    her_template = her_template.replace("[[historic_period_consumer_values]]", str(consumer_historic_values))   
    her_template = her_template.replace("[[historic_period_consumer_values_neighbours]]", str(consumer_historic_values_neighbours))
    her_template = her_template.replace("[[historic_period_consumer_values_efficient_neighbours]]", str(consumer_historic_values_efficient_neighbours))

    #  energy efficiency tips section
    tips_to_use = random.sample(energy_consumption_tips, 2)
    her_template = her_template.replace("[tip_1-text]", tips_to_use[0])
    her_template = her_template.replace("[tip_2-text]", tips_to_use[1])
    
    # saving .html file
    client_html_filename =  "data_raw/" + str(client) + ".html"
    client_html_file = open(client_html_filename, "w", encoding="utf8")
    client_html_file.write(her_template)
    client_html_file.close()


for client in results["client_id"].unique():
    generate(client)
